In [10]:
import random
import pandas as pd
from collections import Counter, namedtuple
import razdel
import pymorphy2

%matplotlib inline

In [1]:
root = "../../data/raw/gazeta/"

In [2]:
with open(root+"gazeta_train.txt", "r") as r:
    for line in r:
        print(eval(line).keys())
        break

dict_keys(['url', 'text', 'title', 'summary', 'date'])


In [4]:
def read_gazeta_records(file_name, shuffle=True, sort_by_date=False):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r") as r:
        for line in r:
            records.append(eval(line)) # Simple hack
            print(records)
            break
    records = pd.DataFrame(records)
    if sort_by_date:
        records = records.sort("date")
    if shuffle:
        records = records.sample(frac=1)
    return records

In [6]:
train_records = read_gazeta_records(root+"gazeta_train.txt")
val_records = read_gazeta_records(root+"gazeta_val.txt")
test_records = read_gazeta_records(root+"gazeta_test.txt")

[{'url': 'https://www.gazeta.ru/financial/2011/11/30/3852658.shtml', 'text': '«По итогам 2011 года чистый отток может составить примерно $80 млрд, в следующем году — около $20 млрд. При этом мы ожидаем, что со второго полугодия 2012 года начнется приток капитала», — заявил «Интерфаксу» замминистра экономического развития Андрей Клепач. Официальные прогнозы по выводу капитала из России становятся все пессимистичными: еще летом власти полагали, что из страны уйдет не более $35 млрд, в сентябре Минэкономразвития назвал цифру $50 млрд, в начале ноября Центробанк пересмотрел оценку до $70 млрд. Очередное изменение прогноза было ожидаемо: по расчетам Центробанка , за январь — октябрь чистый отток капитала достиг $64 млрд, причем в последние месяцы он ускорился: в сентябре он составил $14 млрд, в октябре — $13 млрд против среднего ежемесячного оттока в $6—8 млрд в первом полугодии. «После октябрьских данных Минэкономразвития вынуждено было изменить оценку, настаивать на $70 млрд означало ожид

In [7]:
def read_gazeta_records(file_name, shuffle=True, sort_by_date=False):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r") as r:
        for line in r:
            records.append(eval(line)) # Simple hack
    records = pd.DataFrame(records)
    if sort_by_date:
        records = records.sort("date")
    if shuffle:
        records = records.sample(frac=1)
    return records

In [8]:
train_records = read_gazeta_records(root+"gazeta_train.txt")
val_records = read_gazeta_records(root+"gazeta_val.txt")
test_records = read_gazeta_records(root+"gazeta_test.txt")

In [9]:
print(train_records["date"].min(), train_records["date"].max())
print(val_records["date"].min(), val_records["date"].max())
print(test_records["date"].min(), test_records["date"].max())

2010-06-01 10:35:49 2019-05-31 23:56:26
2019-06-01 08:30:00 2019-09-30 23:11:23
2019-10-01 08:23:02 2020-03-23 22:16:23


In [12]:
USEFUL_POS = ['NOUN', 'ADJF', 'ADJS', 'COMP', 'VERB', 'INFN', 'PRTF', 'PRTS', 'GRND', 'ADVB', 'NPRO', 'PRED']

Stats = namedtuple("Stats", "vocabulary,lemma_vocabulary,words_counts,unique_words_counts")

In [13]:
def collect_stats(records, lower=True, text_max_words=3000, summary_max_words=100, nrows=1000):
    morph = pymorphy2.MorphAnalyzer()
    
    text_stats = Stats(Counter(),  Counter(), list(), list())
    summary_stats = Stats(Counter(),  Counter(), list(), list())

    def update_record_field_stats(field, stats, max_words):
        words = [word.text for word in razdel.tokenize(field)][:max_words]
        lemmas = [morph.parse(word)[0] for word in words]
        lemmas = [lemma for lemma in lemmas if lemma.tag.POS in USEFUL_POS]
        words = [lemma.word for lemma in lemmas]
        lemmas = [lemma.normal_form for lemma in lemmas]
        stats.vocabulary.update(words)
        stats.lemma_vocabulary.update(lemmas)
        stats.words_counts.append(len(words))
        stats.unique_words_counts.append(len(set(words)))

    for (text, summary) in records[['text', 'summary']].values[:nrows]:
        update_record_field_stats(text, text_stats, text_max_words)
        update_record_field_stats(summary, summary_stats, summary_max_words)
    return text_stats, summary_stats

In [14]:
%%time

test_text_stats, test_summary_stats = collect_stats(test_records)
print("Test texts vocabulary size: ", len(test_text_stats.vocabulary))
print("Test texts lemma vocabulary size: ", len(test_text_stats.lemma_vocabulary))
print("Test summaries vocabulary size: ", len(test_summary_stats.vocabulary))
print("Test summaries lemma vocabulary size: ", len(test_summary_stats.lemma_vocabulary))
print("Test common lemmas summary vs text: ", len(set(test_text_stats.lemma_vocabulary.keys()) & set(test_summary_stats.lemma_vocabulary.keys())))

Test texts vocabulary size:  67564
Test texts lemma vocabulary size:  27713
Test summaries vocabulary size:  14379
Test summaries lemma vocabulary size:  7896
Test common lemmas summary vs text:  7627
